In [ ]:
def load_image(image_path):
    image_path = os.path.join(BASE_IMAGE_ADDRESS, image_path)
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels= IMG_DEEP)
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = image / 255.0
    return image

In [ ]:
def get_last_unique_labels(labels, n=2):
    unique_labels = []
    for label in reversed(labels):
        if label not in unique_labels:
            unique_labels.append(label)
        if len(unique_labels) == n:
            break
    return unique_labels

In [ ]:
def split_labels_and_images(images, labels):
    target_labels = get_last_unique_labels(labels)
    group1_labels, group1_images = [], []  
    group2_labels, group2_images = [], []  

    for lbl, img in zip(labels, images):
        if lbl in target_labels:
            group2_labels.append(lbl)
            group2_images.append(img)
        else:
            group1_labels.append(lbl)
            group1_images.append(img)

    return (group1_images, group1_labels), (group2_images, group2_labels)

In [ ]:
def generate_random_negative_index(current_index, upper_limit, unseen_labels):
    filter_list = set(unseen_labels)
    filter_list.add(current_index)
    
    candidate_num = [i for i in range(1, upper_limit + 1) if i not in filter_list]
    return np.random.choice(candidate_num)

generate_random_negative_index(50, 100, [73,62])

In [ ]:
from tqdm import tqdm

def create_pairs(images, labels, unseen_labels):
    pairs = []
    pair_labels = []
    unique_lables = np.unique(labels)

    for idx, image_name in tqdm(enumerate(images), total=len(images)):
        current_label = labels[idx]
        num_classes = len(unique_lables)
        
        #Anchor
        image_anchor = load_image(image_name)

        # Positive pair
        pos_idx = np.random.choice(np.where(labels == current_label)[0])
        pos_image = load_image(images[pos_idx])
        pairs.append([image_anchor, pos_image])
        pair_labels.append(1)

        # Negative pair
        neg_label = generate_random_negative_index(current_label, num_classes, unseen_labels)
        neg_idx = np.random.choice(np.where(labels == neg_label)[0]) 
        neg_image = load_image(images[neg_idx])
        pairs.append([image_anchor, neg_image])
        pair_labels.append(0)

    return np.array(pairs), np.array(pair_labels)

In [ ]:
from tqdm import tqdm
import numpy as np
def create_pairs_lazyLoad(images, labels, unseen_labels):
    pairs = []
    pair_labels = []
    unique_lables = np.unique(labels)

    for idx, image_name in tqdm(enumerate(images), total=len(images)):
        current_label = labels[idx]
        num_classes = len(unique_lables)
        
        #Anchor
        image_anchor = image_name

        # Positive pair
        pos_idx = np.random.choice(np.where(labels == current_label)[0])
        pos_image = images[pos_idx]
        pairs.append([image_anchor, pos_image])
        pair_labels.append(1)

        # Negative pair
        neg_label = generate_random_negative_index(current_label, num_classes, unseen_labels)
        neg_idx = np.random.choice(np.where(labels == neg_label)[0]) 
        neg_image = images[neg_idx]
        pairs.append([image_anchor, neg_image])
        pair_labels.append(0)

    return np.array(pairs), np.array(pair_labels)

In [ ]:
images = os.listdir('102flowers')
text_lables = pd.read_csv('Oxford-102_Flower_dataset_labels.csv')
matlabels = loadmat('imagelabels.mat')
labels = matlabels['labels'][0]

seenList, unseendList = split_labels_and_images(images, labels)

pairs, pair_labels = create_pairs_lazyLoad(seenList[0], seenList[1], np.unique(unseendList[1]))

In [ ]:
print(len(pairs))
print(len(pair_labels))
print(pairs.shape)
print(pair_labels.shape)